In [4]:
#### FIRST ATTEMPT TO DO THE GEOJSON

import geopandas as gpd

# Check the georeference stystem
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Chile/LasTablas_Fire/Progressoes_LasTablas_v3_COM_GOES.shp')
print(gdf.crs)

# gdf = gdf.to_crs(epsg=4326)
# print(gdf.crs)

# If the result is EPSG:32719 -> Coordenate System iks WGS84 / UTM zone 19S -> NO REPROJECTION NEEDED!!!

EPSG:32719


In [1]:
import geopandas as gpd
import pandas as pd

# Leer el archivo y asegurarse que está en CRS métrico adecuado (ejemplo UTM zona 19 Sur para Chile)
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Chile/ColonelFires/Progression_ColonelFire_UTC.shp')
gdf = gdf.to_crs(epsg=32719)  # Usar el epsg adecuado para tu zona

# Renombrar columnas y limpiar atributos
gdf = gdf.rename(columns={'date_hour': 'iso_timestamp'})
gdf = gdf.drop(['source', 'fire', 'type'], axis=1)

# Calcular área y perímetro originales (opcional, los atributos progresivos serán los clave)
gdf['area_ha'] = gdf.geometry.area / 10000
gdf['perimeter_m'] = gdf.geometry.length

# Ordenar por fecha
gdf = gdf.sort_values('iso_timestamp').reset_index(drop=True)

# Disolver polígonos originales por 'iso_timestamp'
agg_dict = {}
for col in gdf.columns:
    if col in ['geometry', 'iso_timestamp']:
        continue
    if pd.api.types.is_numeric_dtype(gdf[col].dtype):
        agg_dict[col] = 'sum'
    else:
        agg_dict[col] = 'first'

gdf_2 = gdf.dissolve(by='iso_timestamp', aggfunc=agg_dict).reset_index()
gdf_2['geometry'] = gdf_2.geometry.buffer(0)  # Limpiar geometrías

# Ordenar para asegurar el orden progresivo
gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)


# Crear geometrías acumuladas, áreas y perímetros progresivos
from shapely.ops import unary_union

geom_union = None
acum_rows = []

for idx, row in gdf_2.iterrows():
    if geom_union is None:
        geom_union = row.geometry
    else:
        geom_union = unary_union([geom_union, row.geometry])
    # Disuelve por si quedan multipolígonos separados
    geom_union = unary_union(geom_union)
    area_acum = geom_union.area / 10000
    perim_acum = geom_union.length
    acum_rows.append({
        'id': idx + 1,
        'order_field': idx + 1,
        'iso_timestamp': row['iso_timestamp'],
        'geometry': geom_union,
        'area_ha': area_acum,
        'perimeter_m': perim_acum
    })

gdf_acum = gpd.GeoDataFrame(acum_rows, crs=gdf_2.crs)


# Convertir a WGS84 solo para exportar y compatibilidad con QGIS
gdf_acum = gdf_acum.to_crs(epsg=4326)
gdf_acum.to_file('chile_202302_colonel_v1.geojson', driver='GeoJSON')


In [2]:
# EDIT SHAPEFILE WITH ISOCHRONES # Las Maquinas - Chile
import geopandas as gpd
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Chile/Las_Maquinas/Progression_Las_Maquinas_UTM19S.shp')

# Rename attributes
gdf = gdf.rename(columns={
    'date_hour': 'iso_timestamp'
})

# Delete attribute
gdf = gdf.drop('area', axis=1)
gdf = gdf.drop('source', axis=1)
gdf = gdf.drop('distance', axis=1)
gdf = gdf.drop('duration', axis=1)
gdf = gdf.drop('ROS', axis=1)
gdf = gdf.drop('gr_rate', axis=1)
gdf = gdf.drop('type', axis=1)
gdf = gdf.drop('Area_QGIS', axis=1)

## Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

# Sort ids with time stamps -> from early to later
gdf = gdf.sort_values('iso_timestamp')

# Prepare agg dict: sumar numéricos, 'first' para el resto
agg_dict = {}
for col in gdf.columns:
    if col in ['geometry', 'iso_timestamp']:
        continue
    if pd.api.types.is_numeric_dtype(gdf[col].dtype):
        agg_dict[col] = 'sum'
    else:
        agg_dict[col] = 'first'

# Convine geometries con la misma 'iso_timestamp'
gdf_2 = gdf.dissolve(by='iso_timestamp', aggfunc=agg_dict)

# devolver iso_timestamp a columna (dissolve lo pone como índice)
gdf_2 = gdf_2.reset_index()

# Recalcular área y perímetro sobre las geometrías combinadas (área de la UNIÓN)
gdf_2['area_ha'] = gdf_2.geometry.area / 10000
gdf_2['perimeter_m'] = gdf_2.geometry.length

# CReate a new 'id' secuancial (starting in 1)
gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

# Crear geometrías acumuladas, áreas y perímetros progresivos
from shapely.ops import unary_union

geom_union = None
acum_rows = []

for idx, row in gdf_2.iterrows():
    if geom_union is None:
        geom_union = row.geometry
    else:
        geom_union = unary_union([geom_union, row.geometry])
    # Disuelve por si quedan multipolígonos separados
    geom_union = unary_union(geom_union)
    area_acum = geom_union.area / 10000
    perim_acum = geom_union.length
    acum_rows.append({
        'id': idx + 1,
        'order_field': idx + 1,
        'iso_timestamp': row['iso_timestamp'],
        'geometry': geom_union,
        'area_ha': area_acum,
        'perimeter_m': perim_acum
    })

gdf_acum = gpd.GeoDataFrame(acum_rows, crs=gdf_2.crs)


# Convertir a WGS84 solo para exportar y compatibilidad con QGIS
gdf_acum = gdf_acum.to_crs(epsg=4326)
gdf_acum.to_file('chile_201701_lasmaquinas_v1.geojson', driver='GeoJSON')




In [3]:
# EDIT SHAPEFILE WITH ISOCHRONES # Las Tablas- Chile
import geopandas as gpd
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Chile/LasTablas_Fire/Progressoes_LasTablas_v3_COM_GOES.shp')

# Rename attributes
gdf = gdf.rename(columns={
    'utc_date': 'iso_timestamp'
})

# Delete attribute
gdf = gdf.drop('area_ha', axis=1)
gdf = gdf.drop('source', axis=1)
gdf = gdf.drop('distance', axis=1)
gdf = gdf.drop('duration', axis=1)
gdf = gdf.drop('ros_mh', axis=1)
gdf = gdf.drop('type', axis=1)
gdf = gdf.drop('grate', axis=1)
gdf = gdf.drop('Area_QGIS', axis=1)

## Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

# Sort ids with time stamps -> from early to later
gdf = gdf.sort_values('iso_timestamp')

# Prepare agg dict: sumar numéricos, 'first' para el resto
agg_dict = {}
for col in gdf.columns:
    if col in ['geometry', 'iso_timestamp']:
        continue
    if pd.api.types.is_numeric_dtype(gdf[col].dtype):
        agg_dict[col] = 'sum'
    else:
        agg_dict[col] = 'first'

# Convine geometries con la misma 'iso_timestamp'
gdf_2 = gdf.dissolve(by='iso_timestamp', aggfunc=agg_dict)

# devolver iso_timestamp a columna (dissolve lo pone como índice)
gdf_2 = gdf_2.reset_index()

# Recalcular área y perímetro sobre las geometrías combinadas (área de la UNIÓN)
gdf_2['area_ha'] = gdf_2.geometry.area / 10000
gdf_2['perimeter_m'] = gdf_2.geometry.length

# CReate a new 'id' secuancial (starting in 1)
gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

# Crear geometrías acumuladas, áreas y perímetros progresivos
from shapely.ops import unary_union

geom_union = None
acum_rows = []

for idx, row in gdf_2.iterrows():
    if geom_union is None:
        geom_union = row.geometry
    else:
        geom_union = unary_union([geom_union, row.geometry])
    # Disuelve por si quedan multipolígonos separados
    geom_union = unary_union(geom_union)
    area_acum = geom_union.area / 10000
    perim_acum = geom_union.length
    acum_rows.append({
        'id': idx + 1,
        'order_field': idx + 1,
        'iso_timestamp': row['iso_timestamp'],
        'geometry': geom_union,
        'area_ha': area_acum,
        'perimeter_m': perim_acum
    })

gdf_acum = gpd.GeoDataFrame(acum_rows, crs=gdf_2.crs)


# Convertir a WGS84 solo para exportar y compatibilidad con QGIS
gdf_acum = gdf_acum.to_crs(epsg=4326)
gdf_acum.to_file('chile_202402_lastablas_v1.geojson', driver='GeoJSON')

